In [ ]:
#Finished strategy

In [ ]:
###########################################################
# ALGORITHM FRAMEWORK V1.1 - 11/08/15 - MARCUS WILLIAMSON #
###########################################################
# PLEASE SEE EVERNOTE PSEUDOCODE FOR MORE DETAILS ON CODE #
###########################################################

'''
MISSING:
- Risk management example logic
- Abilty to pickup existing open positions on algorithm restart
'''

#import libraries
import math
import pandas
#etc..

################################### [1] INITIALIZATION ########################################

'''
This is called when algorithm is first switched on defines fixed aspects in algos operation placeholder security sometimes defined before future csv import can import csv list of securities here but is ONE OFF operation
'''
def initialize(context):
    
    set_commission(commission.PerShare(cost=0.013, min_trade_cost=1.3)) #define commission
       
    context.securities = [sid(23921), sid(12915), sid(24705), sid(19920)..] #define universe
    
    #STRATEGY 1 DEFINED SECURITIES AND INPUTS
    inputs1 = [sid(23921), sid(12915), sid(24705)]
    
    #STRATEGY 2 DEFINED SECURITIES AND INPUTS
    inputs2 = [sid(23921), sid(19920), sid(24705)]
    
    #...
    
    #STRATEGY N DEFINED SECURITIES AND INPUTS
    inputsN = [sid(23921), sid(19920), sid(24705)]
    
    
    #INITIALIZATION OF MODULES ON FIRST RUN
    try: 
        context.first_time #if this is the first time will error onto except
        
    except:
        context.first_time = False #no longer first time 
        
        #DEFINING KEY VARIABLES
        thres_rebal = 0.1 #defining a target vs existing position difference threshold
        
        w1 = 0.3 #portfolio allocations for each alpha generator
        w2 = 0.3
        #...
        wN = 0.3
        
        #DEFINING PORTFOLIO MANAGER
        context.p_manager = PortfolioManagerWeighted(thres = thres_rebal)
        
        #DEFINING EXECUTION HANDLER
        context.exec_handler = ExecutionHandlerMarket()
        
        #DEFINING RISK MANAGER
        context.r_manager = RiskManagerPortfolio()
        
        #DEFINING ALPHAGENERATORS
        context.alpha_1 = AlphaGeneratorStrat1(inputs1)
        context.alpha_2 = AlphaGeneratorStrat2(inputs2)
        #...
        context.alpha_N = AlphaGeneratorStrat2(inputsN)
        
        #ADDING ALPHAGENERATORS TO PORTFOLIO MANAGER
        context.p_manager.add_alpha_generator(context.alpha_1, w1)
        context.p_manager.add_alpha_generator(context.alpha_2, w2)
        #...
        context.p_manager.add_alpha_generator(context.alpha_N, wN)


################################### [2] SCHEDULING ############################################

'''
THIS SECTION IS STILL PART OF INITIALIZE: All main operations of the algorithm that occur during the market opening times are triggered with time based rules instead of manual comparisons taking place every minute bar, they are roughly broken down into housekeeping and strategies, housekeeping ensures records are kept of portfolio metrics and orders are occuring correctly. Strategy timings are based on the requirements of the particular algorithm. Strategies that are run every minute bar are barking up the wrong tree, however they can be placed in handle data...
'''
    #HOUSEKEEPING  
    schedule_function( record_daily_values ,date_rules.every_day(), #bookeeping
                      time_rules.market_close(hours=0,minutes=5))
    schedule_function( check_orders ,date_rules.every_day(), #error checking orders
                      time_rules.market_close(hours=0,minutes=5))

    
    #STRATEGY TIMINGS
    schedule_function( alpha_strat1 ,time_rule=time_rules.market_open(minutes=1))
    
    schedule_function( alpha_strat2 ,date_rule=date_rules.month_start(days_offset=1),
                      time_rule=time_rules.market_open(),half_days=True)
    
    #...
    
    schedule_function( alpha_stratN ,time_rule=time_rules.market_open(minutes=1))

    # END INITIALIZE 
    
################################### [3] BEFORE OPEN ###########################################

'''
Anything in here will be executed the night before the market opens, this is some time in the early hours of the morning, importing of CSV's normally happens here, also if using dynamic signals this is where they are introduced for rest of the algorithm during the trading day
'''
def before_trading_start(context):
    
    fetch_csv( 'url' , date_format='%D-%M-%Y', symbol = 'xyz' )
    #insert logic here to bring new information into algorithm when market opens
    
    pass

################################ [4] STRATEGY HANDLER #########################################

'''
This section contains the strategy functions which are run during market open hours on triggering by a scheduled function. The strategies are activated at specific times during the day this section is stackable
'''
#STRATEGY 1 EVENT 
def alpha_strat1(context, data):
    
    last_rebal   = context.alpha_1.rebalancing_date #last time it was rebalanced
    period       = context.alpha_1.period #period as defined for rebalancing
    current_date = pandas.Period(get_datetime(),freq='D') #current date
    
    if last_rebal is None or current_date >= last_rebal + period: #if hasnt been rebalanced
        context.p_manager.alpha_rebalance[context.alpha_1] = True #flag to be rebalanced
    
    activate_compute_target(context, data) #trigger computing of new targets

#STRATEGY 2 EVENT 
def alpha_strat2(context, data):
    
    context.p_manager.alpha_rebalance[context.alpha_2] = True #flag to be rebalanced
    
    activate_compute_target(context, data) #trigger computing of new targets

    
#...    
    
    
#STRATEGY N EVENT 
def alpha_stratN(context, data):
    
    context.p_manager.alpha_rebalance[context.alpha_N] = True #flag to be rebalanced
    
    activate_compute_target(context, data) #trigger computing of new targets
    

################################### [5] EVENT HANDLER ######################################## 

'''
This is where it all comes together, when the scheduled function has run for the specific alpha strategy it will call this function to compute target. This calls the portfolio manager to promp the alpha generator for what it wants allocated to its strategy then this is tied together into a target portfolio and sent to the risk manager who then examines and modifies sizing if necessary, then passes it onto the market execution handler to action order
'''
#ORDER PROCESSING
def activate_compute_target(context, data):
    
    context.p_manager.compute_target(context, data) #compute new target port in port manager
    
    pretarget = context.p_manager.target_portfolio #initial target portfolio is generated
    
    context.r_manager.compute_risk(context, data, pretarget) #examining risk of target port
    
    target = context.r_manager.risked_target_portfolio #returned risk adjusted target port
    
    context.exec_handler.compute_orders(context, data, target) #send target to execution man

    
'''
The handle data function is called every bar of data      
'''
def handle_data(context, data):
    #if strategy requires to run on every bar of data it can be placed within here or called
    pass
    
    
#RECORDING DAILY METRICS    
def record_daily_values(context,data):
    
    #keeping track of leverage postions and any other data that is relevant to daily operation
    record(leverage = context.account.leverage)
    record(market_exposure = context.account.net_leverage)

    longs = shorts = 0
    for position in context.portfolio.positions.itervalues():
        if position.amount > 0:
            longs += 1
        if position.amount < 0:
            shorts += 1
            
    record(long_count=longs, short_count=shorts)  

'''
This function is called at the end of the trading day and needs to compare the target portfolio that was executed against the actual open orders and any that have failed. If there is a mismatch then a contingency needs to be planned, calling the relevant pair arm to close or to trigger a full portfolio rebalance in order to shore up positions. Closing incorrect positions is top priority as uncalculated exposure could lead to nasty concequences
'''
#CHECKING ORDERS    
def check_orders(context,data):
    
    #Check that actual positions match what is expected by portfolio manager
    #check that portfolio.positions matches target_portfolio!!
    
    pass
      
    
################################## [6] CLASSES TO IMPLEMENT ###################################

'''
These classes to implement represent classes that later aspects of the code call when different strategies require the usage of them. They provide structure and a framework for iterative usage. AlphaGenerator 'abstract class' keeps track of positions and allocations requested, it serves as the class from which the individual alpha generators inherit python dictionaries and functions, so that they can be called in an iterative way later in the program. In other words it provides backbone structure to function which later takes varying inputs
'''
#ALPHA GENERATOR CLASS
class AlphaGenerator(object):

    def __init__(self):
        
        self.alloc = dict() # allocation wanted
        
        #self.pos = dict() # positions associated to the strategy, undecided if this is needed
        
    def compute_allocation(self, context, data):
        
        raise NotImplementedError() #if there is attempt to override this method then error    
   

'''
Portfolio manager module manages the portfolio on the chosen frequency of strategy. It has alpha generators under control that get edge and ask for allocation. It then compute a target allocation being helped by a risk manager before sending to execution handler
'''
#PORTFOLIO MANAGER
class PortfolioManager(object):

    def __init__(self):
        
        self.target_portfolio = dict() #target_portfolio
        
        self.list_alpha = [] #list of strategies
        
        self.alpha_rebalance = dict() #keep track if we want to get new signals from alpha generators
        
    
    def add_alpha_generator(self, alpha_generator): #add a new alpha generator to portfolio
        
        self.list_alpha.append(alpha_generator) #append it to list
        
        self.alpha_rebalance[alpha_generator] = False #rebalanced flag set 

    
    def compute_target(self, context, data): #computes the target portfolio
        
        raise NotImplementedError() 


##################################### [7] ALPHA STRATEGIES ####################################

'''
All alpha generators go here, this can be singular or as many as is desired, an alpha generator can also be used to decide portfolio allocations of the other strategies by consulting self.list_alpha.append and then adjusting the associated weights. This for example could be modfying trend following bias to mean reversion bias in portfolio weighting. This is stackable!!
'''
#ALPHA GENERATOR STRATEGY 1 CLASS #########################
class AlphaGeneratorStrat1(AlphaGenerator):
    
    def __init__(self, inputs1, buy_count=1, leverage=1.45, period=21,lookback=63): #define
        
        AlphaGenerator.__init__(self) #utilising abstract alphagenerator class definitions 
        
        #DEFINING TOP LEVEL VARIABLES OFF OF INPUT 
        self.inputs1 = inputs1 #stocks  
        self.buy_count = buy_count #number of stocks to buy 
        self.buy_percent = leverage / buy_count  
        
        self.rebalancing_date = None #Date of last rebalancing
        self.period = period #Rebalancing period in calendar days
        self.lookback = lookback #The 3-month lookback period
        
    #CUSTOM FUNCTIONS #############       
    def get_best_stock(self, prices, context):  
          
        pct_change = prices.pct_change() #gets rolling 60 day return
        rolling_returns = pandas.rolling_apply((1+pct_change),self.lookback,
                                               lambda x : x.prod()) - 1  
         
        returns_last = rolling_returns.iloc[-1][self.inputs1] #get last 60 day return 
        returns_rank = returns_last.rank(pct=True)  #gets percentile return between (0,1)
        returns_rank.sort() #sorts by returns rank
        
        return returns_rank.index[-self.buy_count:] #Returns highest stocks

    #RESET ALLOCATIONS
    def reset_alloc(self): #resetting stock allocations in py dict
        
        for stock in self.alloc:
            self.alloc[stock] = 0 #set allocation to zero
            
            
    #COMPUTE ALLOCATIONS DESIRED
    def compute_allocation(self, context, data):
        
        histo_range = self.lookback * 2
        histo = history(histo_range, '1d', 'price').dropna()
        
        new_stocks  = self.get_best_stock(histo, context) #next stocks to hold 
        
        if len(new_stocks) > 0: #if there are stocks to buy
            
            self.reset_alloc() #set allocation of all stocks back to zero
            
            for stock in new_stocks:
                
                mavg_50  = data[stock].mavg(50)
                mavg_200 = data[stock].mavg(200)
                
                if mavg_50 > mavg_200: # get allocation only if sma 50 above sma 200
                    
                    self.rebalancing_date = pandas.Period(get_datetime(),freq='D') 
                    #record date of rebalancing
                    self.alloc[stock] = self.buy_percent #put in allocation request

                    
             
#ALPHA GENERATOR STRATEGY 2 CLASS #########################
class AlphaGeneratorStrat2(AlphaGenerator):
    
    def __init__(self, inputs2):
        
        AlphaGenerator.__init__(self) #utilising abstract alphagenerator class definitions 
        
        self.inputs2 = inputs2 #defining from inputs
        
    #COMPUTING ALLOCATION DESIRED
    def compute_allocation(self, context, data):
        
        self.alloc = self.inputs2[get_datetime().month] #requesting its allocation
     
    
#...        


#ALPHA GENERATOR STRATEGY N CLASS #########################
class AlphaGeneratorStratN(AlphaGenerator):
    
    def __init__(self, inputsN):
        
        AlphaGenerator.__init__(self) #utilising abstract alphagenerator class definitions 
        
        self.inputsN = inputsN #defining from inputs
        
    #COMPUTING ALLOCATION DESIRED
    def compute_allocation(self, context, data):
        
        self.alloc = self.inputsN[get_datetime().month] #requesting its allocation
    
    
##################################### [8] PORTFOLIO MANAGER ###################################

'''
Heart and soul of algorithm, everything passes through the portfolio manager in some form, alpha generators are added into the portfolio and it manages the allocations requested from each alpha generator it computes targets portfolios which are passed to risk manager to evaluate before they are sent to execution handler
'''
#PORTFOLIO MANAGER CLASS
class PortfolioManagerWeighted(PortfolioManager):

    def __init__(self, thres):
        
        PortfolioManager.__init__(self) #utilising abstract port manager class definitions 
        
        self.list_weight_alpha = dict() #defining
        
        self.thres = thres #reassigning threshold from initialise
        
        
    #ADD ALPHA GENERATORS
    def add_alpha_generator(self, alpha_generator, weight): 
        #add new alpha generator to port
        
        self.list_alpha.append(alpha_generator) #append
        
        self.list_weight_alpha[alpha_generator] = weight #give it input weighting
        
        self.alpha_rebalance[alpha_generator]   = False #set rebalance flag false

        
    #COMPUTE TARGETS 
    def compute_target(self, context, data):

        self.get_new_alpha_allocation(context, data) #refresh allocation asked by alpha gens

        self.target_portfolio = dict() #clear target portfolio allocations ready

        for alpha in self.list_alpha: #for each alpha add allocation to the target

            for stock in alpha.alloc:

                alloc_alpha = alpha.alloc[stock] * self.list_weight_alpha[alpha] 
                #adjust stock allocation by strategy allocation so rule is not compromised

                self.target_portfolio[stock] = alloc_alpha 
                #add target allocation to target portfolio

        port_value = context.portfolio.portfolio_value #geting portfolio value
        
        for stock in context.portfolio.positions:
            
            if stock in self.target_portfolio: #check if its in target portfolio first
            
                amount = context.portfolio.positions[stock].amount
            
                price  = data[stock].price
            
                alloc_stock = amount * price / port_value #getting current allocation of stock
            
                if abs(self.target_portfolio[stock] - alloc_stock) < self.thres: #if close enough
                
                    self.target_portfolio[stock] = alloc_stock #no need to change
                else:
                    pass
                
    #REFRESH ALPHA ALLOCATIONS
    def get_new_alpha_allocation(self, context, data):
        
        for alpha in self.list_alpha:
            
            if self.alpha_rebalance[alpha]:
                
                self.alpha_rebalance[alpha] = False
                
                alpha.compute_allocation(context, data)

                
################################### [9] Risk Manager ##########################################

'''
Does what it says on the tin, this can be as simple or complex as desired, so long as it takes the input of a target porfolio and provides an output of a risk adjusted target porfolio in the same format as its input
'''
class RiskManagerPortfolio:

    def compute_risk(self, context, data, target_portfolio):
        
        #examine target portfolio considering market exposure and puported leverage
        #may enforce a scaling factor to shrink positions in order to keep target beta
        #down and volatility minimised
        
        #CHECK FOR BAD DATA PRINTS IN ORDER TICKERS
        for stock in target_porfolio: #iterate through stocks in target_portfolio for errors
            
            prices = history(100, '1d', 'close_price').dropna(axis=1) #get last 100 days of stock price data drop NA's
            
            std = prices[stock].std() #getting std of stock

            if abs(prices[stock][-1] - prices[stock][-2]) > 10 * std:
                
                log("{0}: STD = {1}  Error in price data".format(stock.symbol,std)) #log the error
                #implement logic to remove this from order list and recalculate possibly?
        
        
        self.risked_target_portfolio = target_portfolio #output updated target portfolio
            
            
################################### [10] Execution Handler ####################################

'''
Again very obvious section, this is section actions the requests of the rest of the algorithm and records the order in a log. Unfufilled orders are closed and noted here when computing orders. This may be something that could be enhanced or modified 
'''
class ExecutionHandlerMarket:

    def compute_orders(self, context, data, target_portfolio):
        
        #CLOSING FAILED MARKET ORDERS
        D = get_open_orders() 
        for L in D: 
            for ID in D[L]:
                cancel_order(ID) # cancel order
                log.info("Closing failed order: {0}".format(ID))
        
        for stock in context.securities: #this needs modifying from previous design
            
            if stock in target_portfolio:
                order_target_percent(stock, target_portfolio[stock])
                print("Symbol: {0}, Allocation requested = {1}".format(stock.symbol,target_portfolio[stock]))
            else:
                pass
            
            
###############################################################################################
######################################## END OF CODE ##########################################
###############################################################################################